In [1]:
import sys
import numpy as np
import pandas as pd 
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV
sys.path.append("/home/mindy/Documents/projects/creditCardFraud/scripts/")

In [2]:
from util import *

In [3]:
%load_ext autoreload
%autoreload 2

### import data 

In [4]:
train = pd.read_csv("../processedData/TrainingData_normal.csv")
dev = pd.read_csv("../processedData/DevData_normal.csv")
test = pd.read_csv("../processedData/hold_outset_moreFraud.csv")

### generate train, test, and normal data 

In [5]:
training, norm, test_data, y_test = train_test_dfs(train,dev,test,"Class",0.2,1988)

### One-Class SVMS:
  * Train on normal class performs better 

In [6]:
outlier_ratio = training.Class.value_counts(normalize=True)[1]

In [7]:
svm_norm = OneClassSVM(kernel="rbf", nu=outlier_ratio,gamma="scale") 
svm_norm.fit(norm)

OneClassSVM(nu=0.0017248568105510324)

In [8]:
svm_all = OneClassSVM(kernel="rbf", nu=outlier_ratio,gamma="scale")
svm_all.fit(training.drop("Class",axis=1))

OneClassSVM(nu=0.0017248568105510324)

In [9]:
test_score_norm = svm_norm.decision_function(test_data)
test_score_all = svm_all.decision_function(test_data)

In [10]:
model_results(y_test, -test_score_norm, 0)

[[56751   112]
 [   42    57]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56863
           1       0.34      0.58      0.43        99

    accuracy                           1.00     56962
   macro avg       0.67      0.79      0.71     56962
weighted avg       1.00      1.00      1.00     56962



In [11]:
model_results(y_test, -test_score_all, 0)

[[56774    89]
 [   78    21]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56863
           1       0.19      0.21      0.20        99

    accuracy                           1.00     56962
   macro avg       0.59      0.61      0.60     56962
weighted avg       1.00      1.00      1.00     56962



### SVM works better using normal class only 

### Define custom score

In [28]:
custom_score = make_custom_score()

### Define custom train and test splits

In [29]:
cvSplits, X_train,y_train = makeCustomSplits(training,"Class",5,2018,outlier_ratio)

In [30]:
params = {
        "kernel":["rbf","linear"],
        "gamma":[0.0001,0.00001, "scale"],
        "nu":[outlier_ratio*0.75, outlier_ratio,outlier_ratio*1.25]
        }



In [31]:
CV = GridSearchCV(
                OneClassSVM(),
                params,
                scoring= custom_score,
                refit=False,
                verbose = 1,
                cv=cvSplits,
                n_jobs=20
                )

In [ ]:
CV.fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   29.5s


In [ ]:
df_output = CVResultsOutput(CV.cv_results_,custom_score.keys())

In [ ]:
df_output

### Instead of using best selected estimator using refit, use the output with the best overall tp,fp,fn

In [19]:
best_model = OneClassSVM(kernel="rbf",gamma="scale",nu=outlier_ratio)

In [21]:
best_model.fit(norm)

OneClassSVM(nu=0.0017248568105510324)

In [22]:
best_pred_score = best_model.decision_function(test_data)

In [23]:
model_results(y_test,-best_pred_score,0)

[[56751   112]
 [   42    57]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56863
           1       0.34      0.58      0.43        99

    accuracy                           1.00     56962
   macro avg       0.67      0.79      0.71     56962
weighted avg       1.00      1.00      1.00     56962



### Appears that GridSearchCV in this manner is more efficient than orthogonal search